<a href="https://colab.research.google.com/github/vitorgaspar72/projetoicd/blob/main/pratica_integrada_s1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importando bibliotecas

In [ ]:
import requests 
import numpy as np
import pandas as pd
import datetime
# !pip install pandasql
import pandasql as sql
from bs4 import BeautifulSoup

# Faz a busca de dados na página HTML e refina os dados

In [ ]:


def pegar_dados_eventos():
  r = requests.get('https://nuforc.org/webreports/ndxevent.html')
  html = r.text
  array_html = html.split("<TBODY>")
  linhas_com_sujeira = array_html[1].split("</TBODY>")
  linhas = linhas_com_sujeira[0]

  celulas = linhas.split("<TD>")
  celulas
  a_href_com_sujeira = map(lambda i: i.split('<FONT style=FONT-SIZE:11pt FACE="Calibri" COLOR=#000000>'), celulas)
  a_href_com_sujeira = list(a_href_com_sujeira)

  del(a_href_com_sujeira[0])
  a_href_com_sujeira.pop()

  valor_celulas = map(lambda x: x[0].split("</TD>"), a_href_com_sujeira)
  valor_celulas = list(valor_celulas)

  valor_celulas = map(lambda x: x[0], valor_celulas)
  valor_celulas = list(valor_celulas)

  linhas_uteis = filter(lambda x: "<A " in x , valor_celulas)
  linhas_uteis = list(linhas_uteis)

  link_data_com_sujeira = map(lambda x: x.split("<"), valor_celulas)
  link_data_com_sujeira = list(link_data_com_sujeira)

  return link_data_com_sujeira
  link_data_com_sujeira = filter(lambda x: x[1] is not None, link_data_com_sujeira)
  link_data_com_sujeira = list(link_data_com_sujeira)

  # a_href = list()
  # return a_href_com_sujeira[1]
  # for a in a_href_com_sujeira:
  #   a_href.append(a[1])

  links_com_sujeira = map(lambda x: x.split("<A HREF= "), a_href)

  links = list()
  datas = list()
  link_data = list()
  for l in links_com_sujeira:
    link = l[1].split('>')[0]
    data_mes_ano = l[1].split('>')[1].split('</A')[0]

    data = data_mes_ano.split("/")
    data = datetime.date(int(data[1]), int(data[0]), 1)
    # date = datetime.date(2020, 12, 16)

    # data = datetime.strptime(data_com_dia, "%d/%m/%Y").date()
    link_data.append(dict({
      "link":link,
      "data": data
    }))

  return link_data

link_date = pegar_dados_eventos()
link_date

# Determina o intervalo de anos cadastrados que será feita a busca de dados

In [ ]:
def get_valid_events():
  events = list()
  for a in range(1997, 2018):
    for m in range(1, 13):
      mes = str(m).zfill(2)
      events.append(dict({
        "link": f"ndxe{a}{mes}.html",
        "data": datetime.date(int(a), int(m), 1)
      }))

  return events

link_date = get_valid_events()

# Determina o intervalo de busca na página HTML que será feita a busca de dados

In [ ]:
data_inicio = first_date = datetime.date(1997, 9, 1)
data_fim = first_date = datetime.date(2017, 8, 1)

paginas_intervalo = filter(lambda x: x['data'] >= data_inicio and x['data'] <= data_fim, link_date)
paginas_intervalo = list(paginas_intervalo)


# Obtém os dados do intervalo definido de busca e faz o refinamento dos dados

In [ ]:
def chunks(lst, n):
    """Yield successive n-sized chunks from lst."""
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

def buscar_dados_tabulados(link_data):
  df = {
    'date_time',
    'city',
    'state',
    'country',
    'shape',
    'duration',
    'summary',
    'posted',
    'images'
  }

  df = pd.DataFrame(columns=df)
  lista_dados_padronizados = []

  for l in link_data:
    r = requests.get(f"https://nuforc.org/webreports/{l['link']}")

    soup = BeautifulSoup(r.text, 'html.parser')
    table = soup.find('table')

    linhas = table.findAll("tr")

    data = []
    for row in linhas:
      cells = row.findAll("td")
      cols = [ele.text.strip() for ele in cells]

      if cols != []:
        data.append(list(chunks(cols, 9)))
        break

    for d in data[0]:
      dados_padronizados = {
        'date_time': d[0],
        'city':d[1],
        'state':d[2],
        'country':d[3],
        'shape':d[4],
        'duration':d[5],
        'summary':d[6],
        'posted':d[7],
        'images':d[8] if d[8] != "" else "No"
      }
      lista_dados_padronizados.append(dados_padronizados)

  df = df.append(lista_dados_padronizados, ignore_index = True)

  return df 
  
aparicoes_df = buscar_dados_tabulados(paginas_intervalo)



,summary,shape,state,country,duration,city,images,posted,date_time
0,Strange light inside Lake Monona,Light,WI,USA,5 minutes,Madison,No,3/2/04,9/30/97 22:00
1,"Single light resembling a star, but moving spu...",Light,NS,Canada,8-10 seconds.,Nova Scotia (Canada),No,10/30/06,9/30/97 20:00
2,flying-wing shape outlined by 12-14 lights. Ap...,Triangle,CA,USA,12-15s,San Francisco,No,7/5/99,9/28/97 23:15
3,The Weirdest Thing I Have Ever Seen,Other,SD,USA,30 minutes,Egan,No,2/22/05,9/27/97 23:00
4,A big disk with red and green lights on the ri...,Disk,KY,USA,15 minutes,Crestwood,No,8/5/01,9/27/97 05:00
...,...,...,...,...,...,...,...,...,...
100393,Small light south west of Minneapolis maneuver...,Light,MN,USA,Still going,Corcoran,No,8/4/17,8/1/17 02:45
100394,I was looking out the front windshield and loo...,Other,CA,USA,10 seconds,Moreno Valley,No,8/4/17,8/1/17 02:00
100395,"I was walking my dog about 1am on August 1, 20...",Other,FL,USA,<20 seconds,Bradenton,No,5/9/19,8/1/17 01:00
100396,Glowing flying people . seven of them flying l...,,AR,USA,1 hour,Springdale,No,2/13/20,8/1/17 00:00


# Converte todos os dados obtidos e filtrados do site em CSV

In [ ]:
aparicoes_df.to_csv("aparicoes.csv")

# Quantidade de aparições informadas

In [ ]:
linhasGeradas= aparicoes_df.shape[0]
linhasGeradas 

100398

# Aparições por Estado(EUA) em ordem decrescente

In [ ]:
aparicoesPorEstado= "SELECT state as Estado ,count(state) as Aparicoes from aparicoes_df WHERE state != '' AND country='USA' GROUP BY state ORDER BY aparicoes DESC"
aparicoesPorEstado=sql.sqldf(aparicoesPorEstado,locals())
print(aparicoesPorEstado.to_string(index=False))


# Consulta por Cidades dos EUA(Mínimo de Aparições=10)

In [ ]:
aparicoesPorCidade= "SELECT city as Cidade ,count(city) as Aparicoes from aparicoes_df WHERE state != '' AND country='USA' GROUP BY city ORDER BY aparicoes DESC"
aparicoesPorCidade=sql.sqldf(aparicoesPorCidade,locals())
aparicoesMaioresQueDez= aparicoesPorCidade.query('Aparicoes>=10')
print(aparicoesMaioresQueDez.to_string(index=False))

# Porque Phoenix é a cidade que mais informa aparições?
R= Está perto da Área 51

# Consulta de Aparições no Estado Americano com mais registros(Califórnia, mínimo 10 aparições por cidade)

In [ ]:
aparicoesCalifornia= "SELECT city as Cidade ,count(city) as Aparicoes,shape as FormatoObjeto from aparicoes_df WHERE state = 'CA' AND country='USA'  GROUP BY city ORDER BY aparicoes DESC"
aparicoesCalifornia=sql.sqldf(aparicoesCalifornia,locals())
aparicoesMaioresQueDez= aparicoesCalifornia.query("Aparicoes>=10 & FormatoObjeto !='' ")
print(aparicoesMaioresQueDez.to_string(index=False))

                Cidade  Aparicoes FormatoObjeto
             San Diego        392     Rectangle
           Los Angeles        382      Changing
            Sacramento        244      Changing
              San Jose        223         Other
         San Francisco        197       Chevron
                Fresno        134      Triangle
               Modesto         93         Light
             Riverside         93         Light
            Santa Rosa         92         Light
           Bakersfield         91         Light
               Burbank         86          Oval
      Huntington Beach         79        Sphere
         Santa Barbara         78      Fireball
               Ventura         78         Other
              El Cajon         76       Unknown
               Anaheim         74      Fireball
             Escondido         70         Other
                Irvine         68        Sphere
               Oakland         67        Circle
               Redding         65       